In [12]:
import pandas as pd
import pickle
import sqlite3
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

In [16]:
#loading db
sqlite_db_path = "../student_performance.db"
conn = sqlite3.connect(sqlite_db_path)


query_mat = "SELECT * FROM student_mat"
df_mat = pd.read_sql(query_mat, conn)

query_por = "SELECT * FROM student_por"
df_por = pd.read_sql(query_por, conn)


In [17]:
# combining the two in the db
df = pd.concat([df_mat, df_por], ignore_index=True)


In [7]:
conn.close()

In [18]:
print("Loaded Data Preview (Merged Student Data):")
display(df.head())

df = df.dropna(subset=['G3'])

Loaded Data Preview (Merged Student Data):


,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,6,5,6,6
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,4,5,5,6
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,10,7,8,10
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,2,15,14,15
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,4,6,10,10


In [19]:
# encoding categorical features (since sqlite doesn't play nice)
label_encoders = {}
for col in df.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])
    label_encoders[col] = le

In [11]:
# Normalizing numerical features for future reference (further optimization)

scaler = StandardScaler()
numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns
grade_cols = ['G1', 'G2', 'G3']
numeric_cols = [col for col in df.select_dtypes(include=['int64', 'float64']).columns if col not in grade_cols]
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])


In [ ]:
# Feature Engineering (metrics for what makes students at risk)
df['grade_average'] = (df['G1'] + df['G2'] + df['G3']) / 3
df['attendance_engagement'] = df['absences'] / (df['G1'] + df['G2'] + df['G3'] + 1)
df['low_engagement_flag'] = (df['attendance_engagement'] > df['attendance_engagement'].quantile(0.75)).astype(int)

# Had to create multi-class labels to fit the linear model (this seemed better than binary good/bad)
def categorize_g3(grade):
    if grade < 10:
        return 0  # Low
    elif 10 <= grade < 15:
        return 1  # Medium
    else:
        return 2  # High

df['G3_category'] = df['G3'].apply(categorize_g3)

# Defining features and target variable
X = df.drop(columns=['G3', 'G3_category'])
y = df['G3_category']

# Spliting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

predictions = model.predict()

In [21]:
# Example
# Feature Engineering (first 3 metrics for what makes them at risk)
df['sudden_grade_drop'] = df['G1'] - df['G2']
df['attendance_engagement'] = df['absences'] / (df['G1'] + df['G2'] + df['G3'] + 1)
df['low_engagement_flag'] = (df['attendance_engagement'] > df['attendance_engagement'].quantile(0.75)).astype(int)

# Had to create multi-class labels to fit the linear model (this seamed better than binary good/bad)
def categorize_g3(grade):
    if grade < 10:
        return 0  # Low
    elif 10 <= grade < 15:
        return 1  # Medium
    else:
        return 2  # High

df['G3_category'] = df['G3'].apply(categorize_g3)

# Defining features and target variable
X = df.drop(columns=['G3', 'G3_category'])
y = df['G3_category']

# Spliting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)



In [22]:
# Load Logistic Regression model
with open('../Resources/logistic_regression_model.pkl', 'rb') as file:
    model = pickle.load(file)

model = LogisticRegression()
model.fit(X_train, y_train)

predictions = model.predict(X_test)

print(predictions)


[0 1 1 1 1 1 1 2 2 1 1 2 2 2 0 2 1 0 2 1 0 0 1 2 1 0 1 1 1 1 1 1 1 1 2 1 0
 0 1 1 2 1 2 1 2 1 0 1 1 1 1 1 1 0 0 1 1 1 1 0 0 2 0 2 1 1 2 2 2 2 1 2 1 2
 0 2 1 1 0 2 2 2 1 1 0 1 0 1 1 1 1 0 2 1 1 1 1 0 2 1 1 1 1 1 2 1 2 1 1 0 1
 1 1 1 1 0 1 1 1 2 1 0 2 1 1 1 1 1 0 2 1 1 2 1 1 2 1 2 2 2 1 1 1 2 1 0 0 2
 1 1 1 1 1 1 1 2 1 1 0 2 1 1 2 1 1 2 1 1 1 0 1 1 0 1 1 0 1 1 0 1 1 1 1 1 1
 2 1 1 1 1 0 1 0 0 1 1 0 0 1 0 2 1 0 0 1 1 0 0 0]


c:\Users\hmk81\anaconda3\envs\dev\lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# # Baseline logistic regression model
log_model = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs')
log_model.fit(X_train, y_train)
y_pred_log = log_model.predict(X_test)
log_accuracy = accuracy_score(y_test, y_pred_log)
print("Logistic Regression Model Evaluation:")
print("Accuracy:", log_accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred_log))
print(classification_report(y_test, y_pred_log))

Logistic Regression Model Evaluation:
Accuracy: 0.8660287081339713
Classification Report:
              precision    recall  f1-score   support

           0       0.86      0.83      0.84        46
           1       0.88      0.89      0.89       122
           2       0.83      0.83      0.83        41

    accuracy                           0.87       209
   macro avg       0.86      0.85      0.85       209
weighted avg       0.87      0.87      0.87       209

              precision    recall  f1-score   support

           0       0.86      0.83      0.84        46
           1       0.88      0.89      0.89       122
           2       0.83      0.83      0.83        41

    accuracy                           0.87       209
   macro avg       0.86      0.85      0.85       209
weighted avg       0.87      0.87      0.87       209



In [ ]:
# baseline multi-class Decision Tree model
tree_model = DecisionTreeClassifier(random_state=42)
tree_model.fit(X_train, y_train)
y_pred_tree = tree_model.predict(X_test)
tree_accuracy = accuracy_score(y_test, y_pred_tree)
print("Decision Tree Model Evaluation:")
print("Accuracy:", tree_accuracy)
print("Classification Report:")
print(classification_report(y_test, y_pred_tree))
print(classification_report(y_test, y_pred_tree))


Decision Tree Model Evaluation:
Accuracy: 0.8038277511961722
Classification Report:
              precision    recall  f1-score   support

           0       0.71      0.78      0.74        46
           1       0.84      0.82      0.83       122
           2       0.82      0.78      0.80        41

    accuracy                           0.80       209
   macro avg       0.79      0.79      0.79       209
weighted avg       0.81      0.80      0.80       209

              precision    recall  f1-score   support

           0       0.71      0.78      0.74        46
           1       0.84      0.82      0.83       122
           2       0.82      0.78      0.80        41

    accuracy                           0.80       209
   macro avg       0.79      0.79      0.79       209
weighted avg       0.81      0.80      0.80       209

